In [3]:
from datetime import datetime, timedelta
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

from tldextract import extract
import langid

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)

from newspaper import Article

<ipython-input-3-6e87932399ef>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [16]:
start_date = datetime.strptime('2023-04-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d').date()

In [4]:
us_national_news = pd.read_csv('data/mediacloud_us_national.csv')
print(us_national_news.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21856 entries, 0 to 21855
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ap_syndicated         21856 non-null  bool   
 1   collect_date          21856 non-null  object 
 2   feeds                 0 non-null      float64
 3   guid                  21856 non-null  object 
 4   language              21856 non-null  object 
 5   media_id              21856 non-null  int64  
 6   media_name            21856 non-null  object 
 7   media_url             21856 non-null  object 
 8   metadata              21856 non-null  object 
 9   processed_stories_id  21856 non-null  int64  
 10  publish_date          21856 non-null  object 
 11  stories_id            21856 non-null  int64  
 12  story_tags            21856 non-null  object 
 13  title                 21856 non-null  object 
 14  url                   21856 non-null  object 
 15  word_count         

In [7]:
us_national_news[['language', 'media_name', 'media_url', 'publish_date', 'title', 'url']].tail(20)

,language,media_name,media_url,publish_date,title,url
21836,en,Breitbart,http://www.breitbart.com,2023-06-30 20:13:42,"Trump Vows to Cut Federal Funding to Schools Pushing CRT, Gender Ideology if Elected",https://www.breitbart.com/politics/2023/06/30/trump-vows-to-cut-federal-funding-to-schools-pushing-crt-gender-ideology-if-elected/
21837,en,Upworthy,http://www.upworthy.com,2023-06-30 21:50:05,"Trans influencer, UC grad Dylan Mulvaney says Bud Light didn't support her during backlash",https://biztoc.com/x/60406765de2c4ee0?ref=ff
21838,en,Upworthy,http://www.upworthy.com,2023-06-30 21:36:05,Dylan Mulvaney Speaks Out Against Bud Light for Lack of Support,https://biztoc.com/x/9cc99578f345de4b?ref=ff
21839,en,Rolling Stone,http://www.rollingstone.com,2023-06-30 22:07:19,Trump Threatens to Appoint &#8216;Maybe Even Nine&#8217; Supreme Court Justices if Elected,https://www.rollingstone.com/politics/politics-news/trump-moms-for-liberty-supreme-court-1234782045/
21840,en,CNN,http://www.cnn.com/,2023-06-30 23:07:48.329346,Louisiana Democratic governor vetoes bill banning gender-affirming care for minors,https://www.cnn.com/2023/06/30/politics/louisiana-gender-affirming-care-ban-veto-edwards/index.html
21841,en,Daily Caller,http://dailycaller.com,2023-06-30 23:00:38,‘My 401k Misses You’: Black Woman Pumped To Meet Donald Trump In Philadelphia,https://dailycaller.com/2023/06/30/black-woman-pumped-donald-trump/
21842,en,CNN U.S. Edition,http://us.cnn.com,2023-06-30 22:42:34,Louisiana Democratic governor vetoes bill banning gender-affirming care for minors,https://www.cnn.com/2023/06/30/politics/louisiana-gender-affirming-care-ban-veto-edwards/index.html
21843,en,Jezebel,http://jezebel.com,2023-06-30 09:24:00,The ‘Mommy Goddess’ and the Mass Reporting of Sex Workers on Instagram,https://jezebel.com/sex-workers-mass-reporting-instagram-suspension-1850593878
21844,en,Jezebel,http://jezebel.com,2023-06-30 11:35:00,Supreme Court Uses Bogus Case to Open Pandora's Box of Discrimination,https://jezebel.com/supreme-court-uses-bogus-case-to-open-pandoras-box-of-d-1850547102
21845,en,Jezebel,http://jezebel.com,2023-06-30 13:15:00,Dylan Mulvaney Says Bud Light Hung Her Out to Dry in Textbook Case of Rainbow Capitalism,https://jezebel.com/dylan-mulvaney-says-bud-light-hung-her-out-to-dry-in-te-1850595839


In [27]:
relevant_news = us_national_news[us_national_news['media_name'].isin(['Fox News', 'New York Times'])]
relevant_news = relevant_news[relevant_news['language']=='en']
relevant_news = relevant_news[['media_name', 'publish_date', 'title', 'url']]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 18 to 21834
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   media_name    1029 non-null   object
 1   publish_date  1029 non-null   object
 2   title         1029 non-null   object
 3   url           1029 non-null   object
dtypes: object(4)
memory usage: 40.2+ KB
None


In [28]:
relevant_news.publish_date = pd.to_datetime(relevant_news.publish_date)#.dt.date
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 18 to 21834
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    1029 non-null   object        
 1   publish_date  1029 non-null   datetime64[ns]
 2   title         1029 non-null   object        
 3   url           1029 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 40.2+ KB
None


In [30]:
relevant_news = relevant_news[relevant_news['publish_date'].dt.date >= start_date]
relevant_news = relevant_news[relevant_news['publish_date'].dt.date <= end_date]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 18 to 21834
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    1029 non-null   object        
 1   publish_date  1029 non-null   datetime64[ns]
 2   title         1029 non-null   object        
 3   url           1029 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 40.2+ KB
None


In [32]:
def get_article_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        subtitle = article.meta_description
        text = article.text
    except:
        subtitle = None
        text = None
    return subtitle, text

In [33]:
subtitles, texts = [], []
all_urls = list(relevant_news['url'])
for url in tqdm(all_urls):
    subtitle, text = get_article_from_url(url)
    subtitles.append(subtitle)
    texts.append(text)
relevant_news['subtitle'] = subtitles
relevant_news['text'] = texts
print(relevant_news.info())

100%|██████████| 1029/1029 [09:51<00:00,  1.74it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 18 to 21834
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    1029 non-null   object        
 1   publish_date  1029 non-null   datetime64[ns]
 2   title         1029 non-null   object        
 3   url           1029 non-null   object        
 4   subtitle      1029 non-null   object        
 5   text          1029 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 56.3+ KB
None


In [34]:
relevant_news['media_name'].value_counts()

Fox News          651
New York Times    378
Name: media_name, dtype: int64

In [43]:
subtitles_with_null, texts_with_null = [], []
for s in subtitles:
    if s == '' or s == ' ':
        subtitles_with_null.append(None)
    else:
        subtitles_with_null.append(s)
for t in texts:
    if t == '' or t == ' ':
        texts_with_null.append(None)
    else:
        texts_with_null.append(t)
relevant_news['subtitle'] = subtitles_with_null
relevant_news['text'] = texts_with_null
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 18 to 21834
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    1029 non-null   object        
 1   publish_date  1029 non-null   datetime64[ns]
 2   title         1029 non-null   object        
 3   url           1029 non-null   object        
 4   subtitle      997 non-null    object        
 5   text          1029 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 56.3+ KB
None


In [44]:
relevant_news.to_csv('data/nytimes_foxnews_article_texts_and_info.csv',
                     index=False)